In [41]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio
import statistics

## Proprocessing tabular data from INEGI

In [42]:
df = pd.read_excel('Mortalidad_07.xlsx', skiprows=3) # numero de suicidios de 2010 a 2020 de inegi
censo_2020 = pd.read_csv('inegi_2020_censo.csv') # poblacion total en 2020 del censo de poblacion y vivienda 2020 inegi

In [43]:
censo_2020=censo_2020.dropna().reset_index(drop=True)
censo_2020 = censo_2020.rename(columns={' ':'Entidad'})
censo_2020.head()

,Entidad,Total,Hombres,Mujeres
0,Aguascalientes,"1,425,607","696,683","728,924"
1,Baja California,"3,769,020","1,900,589","1,868,431"
2,Baja California Sur,"798,447","405,879","392,568"
3,Campeche,"928,363","456,939","471,424"
4,Coahuila de Zaragoza,"3,146,771","1,563,669","1,583,102"


In [44]:
df.drop(df.tail(11).index,inplace=True)

In [45]:
for col in df.columns[1:]:
    if len(col)==4:
        df = df.rename(columns={col:col + '_' + df.loc[0,col]})
    elif (col[5:]=='1' or col[5:]=='2'):
        df = df.rename(columns={col:col[:4] + '_' + df.loc[0,col]})

In [46]:
df.drop(df.head(2).index, inplace=True)
df.drop(df.tail(2).index, inplace=True)

In [47]:
df.reset_index(drop=True,inplace=True)

In [48]:
df = df.rename(columns={'Entidad federativa de residencia habitual de la persona fallecida':'Entidad'})

In [49]:
totales = [col for col in df.columns if ('Total' in col or col=='Entidad')]
mujeres = [col for col in df.columns if ('Mujeres' in col or col=='Entidad')]
hombres = [col for col in df.columns if ('Hombres' in col or col=='Entidad')]

In [50]:
for col in df.columns[1:]:
    df[col]=df[col] = pd.to_numeric(df[col])

In [51]:
df.head()

,Entidad,2010_Total,2010_Hombres,2010_Mujeres,2011_Total,2011_Hombres,2011_Mujeres,2012_Total,2012_Hombres,2012_Mujeres,...,2017_Mujeres,2018_Total,2018_Hombres,2018_Mujeres,2019_Total,2019_Hombres,2019_Mujeres,2020_Total,2020_Hombres,2020_Mujeres
0,Aguascalientes,50,41,9,98,81,17,116,95,21,...,25,140,114,26,157,129,28,181,157,24
1,Baja California,107,91,16,149,132,16,113,97,16,...,19,194,164,30,169,143,26,97,87,10
2,Baja California Sur,45,43,2,30,24,6,44,35,9,...,5,57,48,9,59,56,3,67,59,8
3,Campeche,62,53,9,81,71,10,63,48,15,...,11,69,58,11,95,85,10,70,48,22
4,Coahuila de Zaragoza,169,147,22,169,143,26,150,130,20,...,36,216,176,40,261,215,46,273,235,38


In [52]:
df=df.merge(censo_2020, left_on=df.index, right_on=censo_2020.index, how='left')
df = df.drop(['key_0','Entidad_y'], 1)
df = df.rename(columns={'Entidad_x':'Entidad', ' Total':'Total', ' Hombres':'Hombres', ' Mujeres':'Mujeres'})
for col in df.columns[34:]:
    df[col]=df[col].apply(lambda x: x.replace(',', ''))
df[[ 'Total','Hombres','Mujeres']]=df[[ 'Total','Hombres','Mujeres']].astype(int)

In [53]:
df['Tasa_2020'] = round(df['2020_Total']/df['Total']*100000,0)
df['Tasa_H_2020'] = round(df['2020_Hombres']/df['Hombres']*100000,0)
df['Tasa_M_2020'] = round(df['2020_Mujeres']/df['Mujeres']*100000,0)
df[['Entidad','Tasa_2020','Tasa_H_2020','Tasa_M_2020']].head()

,Entidad,Tasa_2020,Tasa_H_2020,Tasa_M_2020
0,Aguascalientes,13.0,23.0,3.0
1,Baja California,3.0,5.0,1.0
2,Baja California Sur,8.0,15.0,2.0
3,Campeche,8.0,11.0,5.0
4,Coahuila de Zaragoza,9.0,15.0,2.0


## Cuantitative Analysis

## Datos anuales

In [54]:
df[totales].describe() # Positive skew

,2010_Total,2011_Total,2012_Total,2013_Total,2014_Total,2015_Total,2016_Total,2017_Total,2018_Total,2019_Total,2020_Total
count,32.000000,32.000000,32.000000,32.000000,32.00000,32.00000,32.00000,32.00000,32.000000,32.000000,32.000000
mean,156.437500,178.500000,173.062500,184.031250,197.40625,200.31250,198.37500,202.15625,202.187500,219.750000,241.500000
std,117.847889,130.903313,133.940029,136.213894,147.14177,148.84186,145.05544,149.46876,147.485688,157.212821,187.482816
min,31.000000,30.000000,44.000000,40.000000,42.00000,44.00000,52.00000,46.00000,51.000000,39.000000,50.000000
25%,60.500000,89.000000,86.500000,100.500000,89.25000,108.25000,98.25000,108.75000,106.500000,117.000000,110.500000
50%,131.500000,149.000000,145.500000,139.500000,168.50000,153.00000,162.00000,162.00000,144.500000,171.500000,190.500000
75%,205.250000,221.000000,195.750000,241.250000,246.25000,252.25000,224.50000,262.50000,247.000000,263.500000,312.000000
max,537.000000,600.000000,598.000000,620.000000,670.00000,667.00000,597.00000,658.00000,630.000000,715.000000,832.000000


In [55]:
df[mujeres].describe() # Positive skew

,2010_Mujeres,2011_Mujeres,2012_Mujeres,2013_Mujeres,2014_Mujeres,2015_Mujeres,2016_Mujeres,2017_Mujeres,2018_Mujeres,2019_Mujeres,2020_Mujeres
count,32.000000,32.00000,32.00000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000
mean,28.750000,34.15625,33.59375,33.593750,39.218750,39.937500,36.968750,38.375000,38.062500,40.218750,44.250000
std,27.514219,30.41552,30.48094,28.971465,33.412123,34.262683,30.838272,33.967489,32.948579,33.356079,37.435536
min,2.000000,4.00000,2.00000,7.000000,4.000000,2.000000,6.000000,4.000000,7.000000,3.000000,8.000000
25%,9.000000,15.25000,15.75000,16.000000,20.250000,19.250000,20.000000,18.750000,19.000000,21.750000,18.750000
50%,21.500000,26.50000,25.50000,23.000000,29.000000,30.000000,26.500000,24.000000,26.000000,27.500000,34.000000
75%,33.000000,38.00000,43.50000,38.500000,46.250000,48.250000,39.250000,45.250000,42.500000,47.500000,56.000000
max,132.000000,140.00000,142.00000,143.000000,163.000000,160.000000,129.000000,142.000000,142.000000,162.000000,184.000000


In [56]:
df[hombres].describe() # Positive skew

,2010_Hombres,2011_Hombres,2012_Hombres,2013_Hombres,2014_Hombres,2015_Hombres,2016_Hombres,2017_Hombres,2018_Hombres,2019_Hombres,2020_Hombres
count,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000
mean,127.687500,144.281250,139.375000,150.375000,158.187500,160.250000,161.343750,163.750000,164.125000,179.531250,197.062500
std,91.736452,101.792716,104.236316,108.532959,115.437579,116.216705,116.070782,117.021366,116.333433,125.262044,151.617928
min,25.000000,24.000000,35.000000,32.000000,35.000000,32.000000,44.000000,41.000000,44.000000,34.000000,32.000000
25%,51.500000,72.500000,64.250000,75.750000,68.250000,85.500000,75.250000,89.750000,86.250000,92.750000,87.000000
50%,106.000000,125.500000,119.500000,117.500000,132.500000,128.000000,137.500000,141.000000,119.500000,142.000000,161.000000
75%,168.250000,179.000000,157.500000,208.250000,209.500000,203.250000,189.250000,210.750000,203.500000,220.250000,267.250000
max,405.000000,459.000000,456.000000,477.000000,507.000000,507.000000,476.000000,524.000000,504.000000,553.000000,648.000000


In [57]:
total_anual = pd.DataFrame(df[totales].sum())
total_anual = total_anual.reset_index()
total_anual = total_anual.rename(columns={'index':'Año',0:'Total'})
total_anual.Año = total_anual.Año.str[:4]

mujer_anual = pd.DataFrame(df[mujeres].sum())
mujer_anual= mujer_anual.reset_index()
mujer_anual = mujer_anual.rename(columns={'index':'Año',0:'Total'})
mujer_anual.Año = mujer_anual.Año.str[:4]

hombre_anual = pd.DataFrame(df[hombres].sum())
hombre_anual= hombre_anual.reset_index()
hombre_anual = hombre_anual.rename(columns={'index':'Año',0:'Total'})
hombre_anual.Año = hombre_anual.Año.str[:4]


In [58]:
anual = total_anual.merge(mujer_anual, left_on='Año', right_on='Año')
anual = anual.rename(columns={'Total_x':'Total', 'Total_y':'Mujeres'})

In [59]:
anual = anual.merge(hombre_anual, left_on='Año', right_on='Año')
anual = anual.rename(columns={'Total_x':'Total', 'Total_y':'Hombres'})

In [60]:
anual.drop(anual.head(1).index, inplace=True)

In [61]:
anual.reset_index(drop=True, inplace=True)

In [62]:
anual[['Total','Mujeres','Hombres']] = anual[['Total','Mujeres','Hombres']].astype(int)

In [63]:
anual

,Año,Total,Mujeres,Hombres
0,2010,5006,920,4086
1,2011,5712,1093,4617
2,2012,5538,1075,4460
3,2013,5889,1075,4812
4,2014,6317,1255,5062
5,2015,6410,1278,5128
6,2016,6348,1183,5163
7,2017,6469,1228,5240
8,2018,6470,1218,5252
9,2019,7032,1287,5745


In [64]:
anual.describe() 

,Total,Mujeres,Hombres
count,11.000000,11.000000,11.000000
mean,6265.363636,1184.363636,5079.181818
std,735.099350,135.329430,605.691641
min,5006.000000,920.000000,4086.000000
25%,5800.500000,1084.000000,4714.500000
50%,6348.000000,1218.000000,5128.000000
75%,6469.500000,1266.500000,5246.000000
max,7728.000000,1416.000000,6306.000000


## Graphic Techniques

## Histogram

In [65]:
fig = go.Figure()

fig.add_trace(go.Histogram(x=df['2010_Total'], name='2010'))
fig.add_trace(go.Histogram(x=df['2015_Total'], name='2015'))
fig.add_trace(go.Histogram(x=df['2020_Total'], name='2020'))
#fig.add_trace(go.Histogram(x=anual.Mujeres, name='Mujeres'))
#fig.add_trace(go.Histogram(x=anual.Hombres, name='Hombres'))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.4)
fig.show()

## Boxplot

In [66]:
entidades = df.Entidad.values

# Show figure
fig = go.Figure()

for value in range(len(entidades)):
    data = df.loc[value, totales].values[1:].astype(int)
    standarized = (data - np.mean(data)) / np.std(data)
    fig.add_trace(go.Box(y=standarized, name=df.loc[value,'Entidad'],
                #marker_color = 'indianred'
                 ))
    
fig.update_layout(title=f"Total Suicides Boxplot per entity (standarized)",title_x=0.5)
fig.update_layout(showlegend=False, height=700)
fig.update_layout(
                yaxis_title='Count',
                    )
#pio.write_html(fig, file='suicidios.html', include_plotlyjs='cdn', full_html=False)
fig.show()


In [67]:
years = [i for i in range(2010,2021)]
medianas = []
colors = px.colors.qualitative.Prism

# Show figure
fig = go.Figure()

for year,color in zip(years,colors):
    data = df['{}_Total'.format(year)]
    #standarized = data = (data - np.mean(data)) / np.std(data)
    fig.add_trace(go.Box(y=data, name=year, boxmean='sd',boxpoints='all', text='median',marker_color=color,line_color=color,
                #marker_color = 'indianred'
                 ))
    
    medianas.append(statistics.median(data))
    

    fig.add_trace(go.Scatter(x=[year], y=[statistics.median(data)],
                            text=round(statistics.median(data)),
                            textposition='bottom center',
                            mode='text+markers',
                            hovertemplate=
                            '<b>Year</b>: %{x}<br>',
                            marker=dict(color=[color])
                            ))
    

"""trace='Total' 
fig.add_trace(go.Scatter(x=years, y=medianas,
                            #text=medianas,
                            #textposition='top center',
                            mode='lines',
                            #hovertemplate=
                            #'<b>Year</b>: %{x}<br>',
                            opacity=0.4
                            ))"""

    
fig.update_layout(title=f"Figure 5. Mexican Suicides Boxplot from 2010 to 2020")
fig.update_layout(showlegend=False, height=700)
fig.update_layout(
                yaxis_title='Number of Suicides per Entity',
                    )
#pio.write_html(fig, file='suicidios_2010_2020.html', include_plotlyjs='cdn', full_html=False)
fig.show()

## Line plot

In [68]:
fig = go.Figure()

traces = ['Total','Hombres','Mujeres']
names = ['Total','Men','Women']
for trace,name in zip(traces,names):
    fig.add_trace(go.Scatter(x=anual.Año, y=anual[trace],
                            text=anual[trace],
                            textposition='top center',
                            mode='lines+markers+text',
                            name=name,
                            hovertemplate=
                            '<b>Year</b>: %{x}<br>',
                            ))


fig.update_layout(title='Figure 1. Total Suicides in Mexico from 2010 to 2020',
                   xaxis_title='Year',
                   yaxis_title='Count',
                   )
                   
#pio.write_html(fig, file='total_suicides_2010_2020.html', include_plotlyjs='cdn', full_html=False)
#print(fig.data[0].hovertemplate)
fig.show()

In [69]:
years = [i for i in range(2010,2021)]
entidades = df.Entidad.values
visible = np.array(entidades)
colors = px.colors.qualitative.Prism

# define traces and buttons at once
traces = []
buttons = []

for value in range(len(entidades)):
    traces.append(go.Scatter(x=years, y=df.loc[value,totales].values[1:].astype(int),
                        text=df.loc[value,totales].values[1:].astype(int),
                        textposition='top center',
                        mode='lines+text+markers',
                        name=df.loc[value,'Entidad'],
                        hovertemplate=
                        '<b>Year</b>: %{x}<br>'+
                        '<b>Count</b>: %{y}',
                        line = dict(color=colors[value%len(colors)]),
                        visible= True if value==30 else False
                        ))
    
    buttons.append(dict(label=df.loc[value,'Entidad'],
                        method="update",
                        args=[{"visible":list(visible==df.loc[value,'Entidad'])},
                            {"title":f"Total Suicides in {df.loc[value,'Entidad']}"}]))

updatemenus = [{"active":30,
                "buttons":buttons,
            }]

# Show figure
fig = go.Figure(data=traces,
                layout=dict(updatemenus=updatemenus))
# This is in order to get the first title displayed correctly
first_title = entidades[30]
fig.update_layout(title=f"Figure 3. Total Suicides in {first_title} from 2010 to 2020",title_x=0.5)
#fig.update_layout(showlegend=True)
fig.update_layout(
                xaxis_title='Year',
                yaxis_title='Count',
                height=500
                    )
#pio.write_html(fig, file='suicidios.html', include_plotlyjs='cdn', full_html=False)
fig.show()

In [70]:
fig = go.Figure()
for year in range(2010,2021):
    fig.add_trace(go.Scatter(x=df.Entidad, y=df['{}_Total'.format(year)],
                        #mode='lines+text+markers',
                        name='{}'.format(year),
                        #text=df['{}_Total'.format(year)],
                        #textposition='top center',
                        hovertemplate=
                        '<b>Entity</b>: %{x}<br>'+
                        '<b>Count</b>: %{y}',
                        ))

fig.update_layout(title='Figure 2. Total Suicides per entity',
                    xaxis_title='Entity',
                    yaxis_title='Count',
                    height=700)

#pio.write_html(fig, file='suicidios_total_entity.html', include_plotlyjs='cdn', full_html=False)
fig.show()

In [71]:
fig = go.Figure()

traces = ['Tasa_2020','Tasa_H_2020','Tasa_M_2020']
names = ['Total', 'Hombres', 'Mujeres']
names_en = ['Total','Men','Women']

for trace,name,name_en in zip(traces,names,names_en):
    fig.add_trace(go.Scatter(x=df.Entidad, y=df[trace],
                            textposition='top center',
                            mode='lines+markers',
                            name=name_en,
                            #hovertemplate=
                            #'<b>Year</b>: %{x}<br>',
                            ))

fig.update_layout(title='Figure 4. Suicides for every 100k inhabitants in 2020',
                   xaxis_title='Entity',
                   yaxis_title='Count',
                   height=600)

fig.add_annotation(
        x='Yucatán',
        y=16,
        xref="x",
        yref="y",
        text="4th Place",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=65,
        ay=-30,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="Red",
        opacity=0.6
        )

fig.add_annotation(
        x='Yucatán',
        y=10,
        xref="x",
        yref="y",
        text="3rd Place",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=65,
        ay=-30,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="Blue",
        opacity=0.6
        )

fig.add_annotation(
        x='Yucatán',
        y=5,
        xref="x",
        yref="y",
        text="1st Place",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=65,
        ay=-20,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="Green",
        opacity=0.6
        )

fig.add_vline(x='Yucatán', line_width=0.8, line_dash="dash", line_color="red", y1=24.5)
#print(fig.data[0].hovertemplate)
#pio.write_html(fig, file='suicidios_100k_entity.html', include_plotlyjs='cdn', full_html=False)
fig.show()

## Chloropeth Map

In [72]:
from mapsmx import MapsMX
import geopandas
import json
import pyproj

In [73]:
state = MapsMX().get_geo('state')
state.head()

,cve_geo_ent,cve_ent,nom_ent,geometry_ent
0,01,01,Aguascalientes,"POLYGON ((2470517.824 1155028.588, 2470552.248..."
1,02,02,Baja California,"MULTIPOLYGON (((1493197.166 1849625.247, 14934..."
2,03,03,Baja California Sur,"MULTIPOLYGON (((1694656.344 1227647.637, 16946..."
3,04,04,Campeche,"MULTIPOLYGON (((3544897.199 946994.621, 354491..."
4,05,05,Coahuila de Zaragoza,"POLYGON ((2469954.193 1978522.993, 2469982.807..."


In [74]:
df = pd.DataFrame(df.merge(state,left_on='Entidad',right_on='nom_ent',how='left'))
gdf = geopandas.GeoDataFrame(df, geometry='geometry_ent')
gdf = gdf.set_index('Entidad')

In [75]:
gdf[['2020_Total','Total','Tasa_2020','Tasa_H_2020','Tasa_M_2020']].head() #select columns to map (focus on 2020)

,2020_Total,Total,Tasa_2020,Tasa_H_2020,Tasa_M_2020
Entidad,,,,,
Aguascalientes,181,1425607,13.0,23.0,3.0
Baja California,97,3769020,3.0,5.0,1.0
Baja California Sur,67,798447,8.0,15.0,2.0
Campeche,70,928363,8.0,11.0,5.0
Coahuila de Zaragoza,273,3146771,9.0,15.0,2.0


In [76]:
gdf = gdf.rename(columns={'2020_Total':'Total Suicides 2020','Total':'Total Population 2020','Tasa_2020':'Total Suicides for every 100k 2020','Tasa_H_2020':'Men Suicides for every 100k 2020',
'Tasa_M_2020':'Women Suicides for every 100k 2020'})

In [77]:
gdf = gdf.to_crs(pyproj.CRS.from_epsg(4326))

In [79]:
"""# Data
cols_dd = ['Total Suicides 2020','Total Population 2020','Total Suicides for every 100k 2020','Men Suicides for every 100k 2020','Women Suicides for every 100k 2020']
# we need to add this to select which trace 
# is going to be visible
visible = np.array(cols_dd)
poly_json = json.loads(gdf.geometry_ent.to_json()) #to json
names = ['Total Suicides','Total Population','Suicides for every 100k', 'Men Suicides', 'Women Suicides']
# define traces and buttons at once
traces = []
buttons = []
for value,name in zip(cols_dd,names):
    traces.append(go.Choropleth(
        geojson=poly_json,
        locations=gdf.index, # Spatial coordinates
        z=gdf[value], # Data to be color-coded
        colorbar_title=name,
        visible= True if value==cols_dd[4] else False))

    buttons.append(dict(label=value,
                        method="update",
                        args=[{"visible":list(visible==value)},
                              {"title":f"Figure 5. {value}"}]))

updatemenus = [{"active":4,
                "buttons":buttons,
               }]


# Show figure
fig = go.Figure(data=traces,
                layout=dict(updatemenus=updatemenus))
# This is in order to get the first title displayed correctly
first_title = cols_dd[4]
fig.update_layout(title=f"Figure 6. {first_title}",title_x=0.5)
fig.update_geos(fitbounds="locations", visible=False)
#pio.write_html(fig, file='suicidios_mapa_inegi.html', include_plotlyjs='cdn', full_html=False)
fig.show()"""

'# Data\ncols_dd = [\'Total Suicides 2020\',\'Total Population 2020\',\'Total Suicides for every 100k 2020\',\'Men Suicides for every 100k 2020\',\'Women Suicides for every 100k 2020\']\n# we need to add this to select which trace \n# is going to be visible\nvisible = np.array(cols_dd)\npoly_json = json.loads(gdf.geometry_ent.to_json()) #to json\nnames = [\'Total Suicides\',\'Total Population\',\'Suicides for every 100k\', \'Men Suicides\', \'Women Suicides\']\n# define traces and buttons at once\ntraces = []\nbuttons = []\nfor value,name in zip(cols_dd,names):\n    traces.append(go.Choropleth(\n        geojson=poly_json,\n        locations=gdf.index, # Spatial coordinates\n        z=gdf[value], # Data to be color-coded\n        colorbar_title=name,\n        visible= True if value==cols_dd[4] else False))\n\n    buttons.append(dict(label=value,\n                        method="update",\n                        args=[{"visible":list(visible==value)},\n                              {"tit